# GEM Reconstruction with LC M001-related Transcriptomics — Statistical Tests for Flux Values

***by Kengo Watanabe***  

In the main Python notebook, the maximum flux values, which were calculated using mouse genome-scale metabolic models (GEMs; Khodaee, S. et al. Sci. Rep. 2020) with the Longevity Consortium (LC) M001-related transcriptomics dataset (Tyshkovskiy, A. et al. Cell Metab. 2019), are compared.  
**–> To maintain the consistency with the DIRAC analyses, statistical tests are performed in this sub-notebook with R kernel.**  

Input:  
* Cleaned flux data: 220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_max-flux-raw-data.tsv  
* Cleaned sample–model metadata: 220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_sample-metadata.tsv  
* Cleaned reaction metadata: 220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_reaction-metadata.xlsx  

Output:  
* Supplementary Data 3  

Original notebook (memo for my future tracing):  
* dalek:[JupyterLab HOME]/220606_LC-M001-related-transcriptomics-GEM/220610_LC-M001-related-transcriptomics-GEM-ver2-4_StatisticalTest.ipynb  

In [ ]:
library("tidyverse")
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

#CRAN
for (package in c("readxl", "dunn.test", "openxlsx")) {
    #install.packages(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}

## 1. Clean the original tables

> Just import the data and metadata cleaned in the main notebook.  

In [ ]:
#Import flux data
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_"
fileName <- "max-flux-raw-data.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t")
print(str_c("nrow: ",nrow(temp)))
head(temp)

flux_data <- temp

In [ ]:
#Import sample-model metadata
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_"
fileName <- "sample-metadata.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t")
print(str_c("nrow: ",nrow(temp)))
head(temp)

sample_meta <- temp

In [ ]:
#Import reaction metadata
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_"
fileName <- "reaction-metadata.xlsx"
sheetName <- "Reaction"
temp <- read_excel(str_c(fileDir,ipynbName,fileName), sheet=sheetName)
print(str_c("nrow: ",nrow(temp)))
head(temp)

rxn_meta <- temp

## 2. Check data structure of the flux values

> See the main notebook

## 3. Compare flux values

> 1. Test the rank of maximum flux values across interventions for each reaction using Kruskal–Wallis H-test.  
> 2. Then, perform post-hoc comparisons between control vs. each intervention using Dunn's test.  
>
> To increase the statistical power, samples are grouped by intervention (i.e., strain, age, and sex are pooled) in the Kruskal–Wallis H-test. Note that Kruskal-Wallis H-test can shrink the variance utilizing all samples (per reaction), whose statistical power is better than the repeated Mann–Whitney U-tests (a.k.a., Wilcoxon rank-sum tests) in the case of small sample size. Although tricky, the P-value adjustment in (1) is performed across all reactions under the assumption that reactions are independent, which would be more conservative and less likely raise referees' eyebrows than using nominal P-value cutoff. Also, the reactions assigned with invariable flux values across all groups are eliminated from tests in advance, which can reduce the number of hypotheses. Because the post-hoc comparisons (2) are to address the effect of each intervention within a specific reaction, the P-values are adjusted across interventions only within the reaction (not across reactions). 

### 3-1. Kruskal–Wallis H-test (Flux ~ Intervention), followed by Dunn's test (Intervention)

#### 3-1-1. Reactions to be eliminated from the tests

> Of note, kruskal.test() fills NA for the case of invariable values, but dunn.test() produces an error.  
> –> Hence, it is needed to eliminate the invariable reactions manually.  

In [ ]:
#Calculate SD per category
temp <- flux_data %>%
    tidyr::gather(key=ModelID, value=Flux, -ReactionID) %>%
    dplyr::left_join(., sample_meta, by="ModelID") %>%
    dplyr::group_by(ReactionID, Category) %>%
    dplyr::summarize(Count=n(), SD=sd(Flux)) %>%
    dplyr::ungroup()
print(str_c("Before:",nrow(temp)))

#Select the reactions assigned with invariable flux values in any category
temp <- temp %>%
    dplyr::filter(SD==0.0)
print(str_c("After:",nrow(temp)))
head(temp)

#Check the reactions assigned with invariable flux values in all categories
temp1 <- temp %>%
    dplyr::group_by(ReactionID) %>%
    dplyr::summarize(nZeros=n()) %>%
    dplyr::ungroup() %>%
    dplyr::filter(nZeros==length(unique(sample_meta$Category)))

#Check the reactions assigned with invariable flux values per category
temp2 <- temp %>%
    dplyr::group_by(Category) %>%
    dplyr::summarize(nZeros=n()) %>%
    dplyr::ungroup()

print(str_c("All reactions in the model: ", nrow(rxn_meta)))
print(str_c("Reactions assigned with flux values: ", nrow(flux_data)))
print(str_c(" -> Invariable in any category: ", length(unique(temp$ReactionID))))
print(str_c(" -> Invariable in all categories: ", nrow(temp1)))
print(str_c(" -> Invariable per category:"))
temp2

rxn_removed <- temp1

#### 3-1-2. Simultaneously perform all Kruskal–Wallis H-tests

In [ ]:
#Prepare DF
group_vec <- c("Con1", "Acar", "17aE", "Prot", "Rapa", "CalR",
               "Con2", "MetR", "GHRw", "GHRk", "SneW", "SneD")
temp <- flux_data %>%
    tidyr::gather(key=ModelID, value=Flux, -ReactionID) %>%
    dplyr::left_join(., sample_meta, by="ModelID") %>%
    dplyr::mutate(Intervention=factor(Intervention, levels=group_vec),
                  Sex=factor(Sex, levels=c("F", "M")),
                  Age=factor(Age, levels=c("05M", "06M", "12M", "14M")))
print(str_c("Before:",nrow(temp)))

#Eliminate the invariable reactions
temp <- temp %>%
    dplyr::filter(!(ReactionID %in% rxn_removed$ReactionID))
print(str_c("After:",nrow(temp)))
head(temp)

#Simultaneously perform all tests using tidyr::nest()
temp <- temp %>%
    dplyr::group_by(ReactionID) %>%
    tidyr::nest() %>%#New column name becomes "data"
    dplyr::mutate(Kruskal=lapply(data, function(tbl) {kruskal.test(Flux~Intervention, data=tbl)})) %>%
    dplyr::ungroup()
print(nrow(temp))
print(head(temp))#print() because Jupyter Lab tries to display list contents

model1 <- temp

In [ ]:
#Check result object
model1$Kruskal[[1]]
str(model1$Kruskal[[1]])

In [ ]:
#Prepare summary table of Kruskal-Wallis H-tests
label <- "Intervention"
temp <- model1 %>%
    dplyr::mutate("{label}_DF":=sapply(Kruskal, function(htest) {unname(htest[["parameter"]])}),
                  "{label}_Hstat":=sapply(Kruskal, function(htest) {unname(htest[["statistic"]])}),
                  "{label}_Pval":=sapply(Kruskal, function(htest) {unname(htest[["p.value"]])})) %>%
    #P-value adjustment with the Benjamini-Hochberg method
    ##Using !!as.name() in the following line, because simple {{}} and !! didn't recognize?
    dplyr::mutate("{label}_AdjPval":=p.adjust(!!as.name(str_c(label,"_Pval")), method="BH")) %>%
    dplyr::select(-data, -Kruskal)

print(str_c("nrow: ",nrow(temp)))
head(temp)

model1_summary <- temp

In [ ]:
#Check
temp1 <- str_subset(names(model1_summary), "Pval")
for (pval in temp1) {
    temp <- model1_summary %>%
        dplyr::filter(!!as.name(pval)<0.05)
    print(str_c(pval," < 0.05: ",nrow(temp)))
}

#### 3-1-3. Simultaneously perform all Dunn's tests

In [ ]:
#Prepare DF
group_vec <- c("Con1", "Acar", "17aE", "Prot", "Rapa", "CalR",
               "Con2", "MetR", "GHRw", "GHRk", "SneW", "SneD")
temp <- flux_data %>%
    tidyr::gather(key=ModelID, value=Flux, -ReactionID) %>%
    dplyr::left_join(., sample_meta, by="ModelID") %>%
    dplyr::mutate(Intervention=factor(Intervention, levels=group_vec),
                  Sex=factor(Sex, levels=c("F", "M")),
                  Age=factor(Age, levels=c("05M", "06M", "12M", "14M")))
print(str_c("Before:",nrow(temp)))

#Eliminate the invariable reactions
temp <- temp %>%
    dplyr::filter(!(ReactionID %in% rxn_removed$ReactionID))
print(str_c("After:",nrow(temp)))
head(temp)

#Simultaneously perform all tests using tidyr::nest()
temp <- temp %>%
    dplyr::group_by(ReactionID) %>%
    tidyr::nest() %>%#New column name becomes "data"
    dplyr::mutate(Dunn=lapply(data, function(tbl) {
        #Save a temporal object to suppress too many output texts from dunn.test()
        invisible(capture.output(output <- as_tibble(dunn.test(tbl$Flux, g=tbl$Intervention,
                                                               method="none", kw=FALSE, label=FALSE,
                                                               wrap=FALSE, table=FALSE, list=FALSE,
                                                               rmc=FALSE, alpha=0.05, altp=FALSE))))
        output})) %>%
    dplyr::ungroup()
print(nrow(temp))
print(head(temp))#print() because Jupyter Lab tries to display list contents

model2 <- temp

In [ ]:
#Check result object
temp <- model2$Dunn[[1]]
print(nrow(temp))
head(temp)
print(str_c("nCombinations: ",length(group_vec)*(length(group_vec)-1)/2))

> –> Comparison labels are based on the matrix of alphabetically ordered column - row, even if setting factor levels in advance...  
> (Note: chi2 is same with the Kruskal–Wallis H-statistic.)  

In [ ]:
#Prepare summary table of Dunn's tests
category_vec <- c("Con1", "Con2", "GHRw", "SneW")#Each control
temp <- model2 %>%
    dplyr::select(ReactionID, Dunn)
for (category in category_vec) {
    #Prepare comparison labels
    temp1 <- sample_meta %>%
        dplyr::mutate(Intervention=factor(Intervention, levels=group_vec)) %>%
        dplyr::arrange(Intervention) %>%
        dplyr::filter(Category==!!category)
    temp1 <- unique(temp1$Intervention)
    baseline <- temp1[1]
    comparison_vec <- c()
    for (i in 1:length(temp1)) {
        contrast <- temp1[i]
        if (contrast!=baseline) {
            comparison <- str_c(contrast,"-vs-",baseline)
            comparison_vec <- c(comparison_vec, comparison)
        }
    }
    #Retrieve and add the results per comparison
    for (i in 1:length(comparison_vec)) {
        label <- comparison_vec[i]
        contrast <- str_split(label, "-vs-", simplify=TRUE)[1]
        baseline <- str_split(label, "-vs-", simplify=TRUE)[2]
        label_f <- str_c(contrast," - ",baseline)
        label_r <- str_c(baseline," - ",contrast)
        temp <- temp %>%
            dplyr::mutate("{label}_zStat":=sapply(Dunn, function(tbl) {
                              if (label_f %in% tbl$comparisons) {
                                  temp1 <- tbl %>%
                                      dplyr::filter(comparisons==label_f)
                                  temp1$Z[1]
                              } else {#Inverse case in Dunn's test
                                  temp1 <- tbl %>%
                                      dplyr::mutate(negZ=-Z) %>%#Flip the sign of statistic
                                      dplyr::filter(comparisons==label_r)
                                  temp1$negZ[1]
                              }}),
                          "{label}_Pval":=sapply(Dunn, function(tbl) {
                              if (label_f %in% tbl$comparisons) {
                                  temp1 <- tbl %>%
                                      dplyr::filter(comparisons==label_f)
                                  temp1$P[1]
                              } else {#Inverse case in Dunn's test
                                  temp1 <- tbl %>%
                                      dplyr::filter(comparisons==label_r)
                                  temp1$P[1]
                              }}),
                          "{label}_AdjPval":=1.0)#Insert dummy column for now
    }
}
temp <- temp %>%
    dplyr::select(-Dunn)

#P-value adjustment across comparisons with the Holm-Bonferroni method per reaction
temp1 <- str_subset(names(temp), "_Pval")
temp2 <- temp %>%
    dplyr::select(ReactionID, all_of(temp1)) %>%
    tidyr::gather(key=ColName, value=Pval, -ReactionID) %>%
    dplyr::group_by(ReactionID) %>%
    dplyr::mutate(AdjPval=p.adjust(Pval, method="holm")) %>%
    dplyr::ungroup() %>%
    dplyr::select(-Pval) %>%
    dplyr::mutate(ColName=str_replace(ColName, "_Pval", "_AdjPval_temp")) %>%
    tidyr::spread(key=ColName, value=AdjPval)

#Replace the dummy values with the true adjusted p-values
temp <- dplyr::left_join(temp, temp2, by="ReactionID")
temp1 <- str_subset(names(temp), "_AdjPval_temp")
for (col_n in temp1) {
    label <- str_replace(col_n, "_AdjPval_temp", "")
    temp <- temp %>%
        ##Using !!as.name() in the following line, because simple {{}} and !! didn't recognize?
        dplyr::mutate("{label}_AdjPval":=!!as.name(str_c(label,"_AdjPval_temp"))) %>%
        dplyr::select(-!!as.name(str_c(label,"_AdjPval_temp")))
}

print(str_c("nrow: ",nrow(temp)))
head(temp)

model2_summary <- temp

In [ ]:
#Check
temp <- str_subset(names(model2_summary), "_AdjPval")
model2_summary %>%
    dplyr::select(ReactionID, all_of(temp)) %>%
    tidyr::gather(key=Comparison, value=AdjPval, -ReactionID) %>%
    dplyr::mutate(Comparison=str_replace(Comparison, "_AdjPval", "")) %>%
    dplyr::filter(AdjPval<0.05) %>%
    dplyr::group_by(Comparison) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup()

temp <- str_subset(names(model2_summary), "_Pval")
model2_summary %>%
    dplyr::select(ReactionID, all_of(temp)) %>%
    tidyr::gather(key=Comparison, value=Pval, -ReactionID) %>%
    dplyr::mutate(Comparison=str_replace(Comparison, "_Pval", "")) %>%
    dplyr::filter(Pval<0.05) %>%
    dplyr::group_by(Comparison) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup()

#### 3-1-4. Save summary tables

> Because non-parametric tests were performed, median and median absolete deviation (MAD) are used as general statistics.  

In [ ]:
#Calculate general statistics
temp <- flux_data %>%
    tidyr::gather(key=ModelID, value=Flux, -ReactionID) %>%
    dplyr::left_join(., sample_meta, by="ModelID") %>%
    dplyr::mutate(Intervention=factor(Intervention, levels=group_vec),
                  Sex=factor(Sex, levels=c("F", "M")),
                  Age=factor(Age, levels=c("05M", "06M", "12M", "14M"))) %>%
    dplyr::group_by(ReactionID, Intervention) %>%
    dplyr::summarize(Count=n(),
                     FluxMedian=median(Flux),
                     FluxMAD=mad(Flux, center=median(Flux), constant=1)) %>%
    dplyr::ungroup()
print(str_c("nrow: ",nrow(temp)))
head(temp)

general_summary <- temp

In [ ]:
#Create a workbook object to save as one single .xlsx file
workbook <- createWorkbook()

#Prepare reaction metadata sheet
sheetName <- "ReactionMetadata"
addWorksheet(workbook, sheetName=sheetName)
writeData(workbook, sheetName, rxn_meta)

#Add the summary table to the workbook object
##Merge tables
temp <- rxn_meta %>%
    dplyr::select(ReactionID, ReactionName) %>%
    dplyr::filter(ReactionID %in% flux_data$ReactionID)#Reactions assigned with flux values
for (group in group_vec) {
    temp <- general_summary %>%
        dplyr::filter(Intervention==!!group) %>%
        dplyr::select(-Intervention) %>%
        dplyr::rename("{group}_N":=Count,
                      "{group}_FluxMedian":=FluxMedian,
                      "{group}_FluxMAD":=FluxMAD) %>%
        dplyr::left_join(temp, ., by="ReactionID")
}
temp <- temp %>%
    dplyr::left_join(., model1_summary, by="ReactionID") %>%#Add NA if not tested (invariable reaction)
    dplyr::left_join(., model2_summary, by="ReactionID") %>%#Add NA if not tested (invariable reaction)
    dplyr::arrange(Intervention_Pval)
print(str_c("nrow: ",nrow(temp)))
head(temp)
##Save the summary table as a new sheet
sheetName <- "Main_vs-each-control"
addWorksheet(workbook, sheetName=sheetName)
writeData(workbook, sheetName, temp)

#Save the workbook as one single .xlsx file
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_StatisticalTest_"
fileName <- "flux-comparison.xlsx"
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

# — †1. Go back to †1 of the main Python notebook —  

## 4. Compare flux values between MR and CR

> Model is same as the above Dunn's test, but post-hoc comparisons are differently retrieved to compare MR vs. CR.  

### 4-1. Another post-hoc Dunn's test

#### 4-1-1. Retrieve the results of target comparisons

In [ ]:
#Prepare summary table of Dunn's tests
comparison_vec <- c("CalR-vs-Con1", "MetR-vs-Con2", "MetR-vs-CalR")
temp <- model2 %>%
    dplyr::select(ReactionID, Dunn)
##Retrieve and add the results per comparison
for (i in 1:length(comparison_vec)) {
    label <- comparison_vec[i]
    contrast <- str_split(label, "-vs-", simplify=TRUE)[1]
    baseline <- str_split(label, "-vs-", simplify=TRUE)[2]
    label_f <- str_c(contrast," - ",baseline)
    label_r <- str_c(baseline," - ",contrast)
    temp <- temp %>%
        dplyr::mutate("{label}_zStat":=sapply(Dunn, function(tbl) {
                          if (label_f %in% tbl$comparisons) {
                              temp1 <- tbl %>%
                                  dplyr::filter(comparisons==label_f)
                              temp1$Z[1]
                          } else {#Inverse case in Dunn's test
                              temp1 <- tbl %>%
                                  dplyr::mutate(negZ=-Z) %>%#Flip the sign of statistic
                              dplyr::filter(comparisons==label_r)
                              temp1$negZ[1]
                          }}),
                      "{label}_Pval":=sapply(Dunn, function(tbl) {
                          if (label_f %in% tbl$comparisons) {
                              temp1 <- tbl %>%
                                  dplyr::filter(comparisons==label_f)
                              temp1$P[1]
                          } else {#Inverse case in Dunn's test
                              temp1 <- tbl %>%
                                  dplyr::filter(comparisons==label_r)
                              temp1$P[1]
                          }}),
                      "{label}_AdjPval":=1.0)#Insert dummy column for now
}
temp <- temp %>%
    dplyr::select(-Dunn)

#P-value adjustment across comparisons with the Holm-Bonferroni method per reaction
temp1 <- str_subset(names(temp), "_Pval")
temp2 <- temp %>%
    dplyr::select(ReactionID, all_of(temp1)) %>%
    tidyr::gather(key=ColName, value=Pval, -ReactionID) %>%
    dplyr::group_by(ReactionID) %>%
    dplyr::mutate(AdjPval=p.adjust(Pval, method="holm")) %>%
    dplyr::ungroup() %>%
    dplyr::select(-Pval) %>%
    dplyr::mutate(ColName=str_replace(ColName, "_Pval", "_AdjPval_temp")) %>%
    tidyr::spread(key=ColName, value=AdjPval)

#Replace the dummy values with the true adjusted p-values
temp <- dplyr::left_join(temp, temp2, by="ReactionID")
temp1 <- str_subset(names(temp), "_AdjPval_temp")
for (col_n in temp1) {
    label <- str_replace(col_n, "_AdjPval_temp", "")
    temp <- temp %>%
        ##Using !!as.name() in the following line, because simple {{}} and !! didn't recognize?
        dplyr::mutate("{label}_AdjPval":=!!as.name(str_c(label,"_AdjPval_temp"))) %>%
        dplyr::select(-!!as.name(str_c(label,"_AdjPval_temp")))
}

print(str_c("nrow: ",nrow(temp)))
head(temp)

model2_summary <- temp

In [ ]:
#Check
temp <- str_subset(names(model2_summary), "_AdjPval")
model2_summary %>%
    dplyr::select(ReactionID, all_of(temp)) %>%
    tidyr::gather(key=Comparison, value=AdjPval, -ReactionID) %>%
    dplyr::mutate(Comparison=str_replace(Comparison, "_AdjPval", "")) %>%
    dplyr::filter(AdjPval<0.05) %>%
    dplyr::group_by(Comparison) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup()

temp <- str_subset(names(model2_summary), "_Pval")
model2_summary %>%
    dplyr::select(ReactionID, all_of(temp)) %>%
    tidyr::gather(key=Comparison, value=Pval, -ReactionID) %>%
    dplyr::mutate(Comparison=str_replace(Comparison, "_Pval", "")) %>%
    dplyr::filter(Pval<0.05) %>%
    dplyr::group_by(Comparison) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup()

#### 4-1-2. Save summary tables

> Because non-parametric tests were performed, median and median absolete deviation (MAD) are used as general statistics.  

In [ ]:
#Load the existing .xlsx file as a new workbook object
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_StatisticalTest_"
fileName <- "flux-comparison.xlsx"
workbook <- loadWorkbook(str_c(fileDir,ipynbName,fileName))

#Add the summary table to the workbook object
##Merge tables
temp <- rxn_meta %>%
    dplyr::select(ReactionID, ReactionName) %>%
    dplyr::filter(ReactionID %in% flux_data$ReactionID)#Reactions assigned with flux values
for (group in c("Con1", "CalR", "Con2", "MetR")) {
    temp <- general_summary %>%
        dplyr::filter(Intervention==!!group) %>%
        dplyr::select(-Intervention) %>%
        dplyr::rename("{group}_N":=Count,
                      "{group}_FluxMedian":=FluxMedian,
                      "{group}_FluxMAD":=FluxMAD) %>%
        dplyr::left_join(temp, ., by="ReactionID")
}
temp <- temp %>%
    #dplyr::left_join(., model1_summary, by="ReactionID") %>%#Add NA if not tested (invariable reaction)
    dplyr::left_join(., model2_summary, by="ReactionID") %>%#Add NA if not tested (invariable reaction)
    dplyr::arrange(`MetR-vs-CalR_Pval`)
print(str_c("nrow: ",nrow(temp)))
head(temp)
##Save the summary table as a new sheet
sheetName <- "Posthoc2_MR-vs-CR"
addWorksheet(workbook, sheetName=sheetName)
writeData(workbook, sheetName, temp)

#Update the existing .xlsx file
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

# — †2. Go back to †2 of the main Python notebook —  

# — Session information —

In [ ]:
sessionInfo()